## Pauli Transfer Matrix extraction
$\def\bra#1{\mathinner{\left\langle{#1}\right|}}\def\ket#1{\mathinner{\left|{#1}\right\rangle}}$

In [1]:
import numpy as np; pi = np.pi
import scipy.io as spio
import sys
import bisect
from qutip import *
from qutip.qip.operations import gate_expand_1toN, gate_expand_2toN

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# Figures should have a width of a 8.6 cm or 3 3/8 in, the width of a single manuscript column.
golden_mean = (np.sqrt(5)-1.0)/2.0 # Aesthetic ratio
fig_width = 3+3/8 # width in inches
fig_height = fig_width*golden_mean # height in inches (0.75 is the standard height to width ratio in Python)
fig_size = [fig_width, fig_height]
mpl.rcParams.update({
    'axes.labelsize' : 12,
    'font.size' : 12,
    'legend.fontsize' : 8,
    'xtick.labelsize' : 10,
    'ytick.labelsize' : 10,
    'ytick.minor.pad' : -0.5,
    'ytick.minor.pad' : -0.5,
    'ytick.major.size' : 1,
    'ytick.minor.size' : 1,
    'ytick.major.width' : .5,
    'ytick.minor.width' : .5,
    'xtick.major.size' : 1,
    'xtick.minor.size' : 1,
    'xtick.major.width' : .5,
    'xtick.minor.width' : .5,
    'xtick.major.pad' : 1.5,
    'xtick.minor.pad' : 1.5,
    'text.usetex' : True,
    'figure.dpi' : 100,
})

#### Parameters

In [2]:
num_lvl = 20 # number of levels
a = destroy(num_lvl) # annihilation operator
K = 1 # kerr amplitude
        def E(t,args):.957
G = 4*K # two photon pump amplitude
alpha = np.sqrt(G/K) # coherent state amplitude

# cat states
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

# computational basis
up = (cat_plus + cat_minus)/np.sqrt(2)
down = (cat_plus - cat_minus)/np.sqrt(2)

# Identity in computational basis
I = up*up.dag() + down*down.dag()

# sigma-z in computational basis
sigma_z = up*up.dag() - down*down.dag()

# sigma-x in computational basis
sigma_x = up*down.dag() + down*up.dag()

# sigma-y in computational basis
sigma_y = 1j*(-up*down.dag() + down*up.dag())

# Array with Pauli matrices
P = [I, sigma_x, sigma_y, sigma_z]

### $R_z(\phi)$-gate
This has the ideal PTM
$$
R =
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & \cos\phi & -\sin\phi & 0 \\
0 & \sin\phi & \cos\phi & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
$$

In [9]:
T_g = 2

# single photon pump amplitude
def E(t,args):
    phi = args['phi']
    return np.pi*phi/(8*T_g*alpha)*np.sin(np.pi*t/T_g)

# Hamiltonian
H0 = - K * pow(a.dag(),2)*pow(a,2) + G * (pow(a.dag(),2) + pow(a,2))
H1 = a.dag() + a
H_tot = [H0,.957[H1,E]]

### The fast way

In [10]:
phi = 0.9046 # angle of rotation
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=15e3, atol=1e-8, rtol=1e-6)

d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for j in range(d**2):
    rho = mesolve(H_tot, P[j], [0,T_g] , c_ops, args = {'phi': phi}, options = opt)
    Lambda = rho.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * np.real((P[i]*Lambda).tr())
R = Qobj(R,dims=[[2],[2]]) # Make quantum object

In [5]:
R

Quantum object: dims = [[2], [2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 9.99999216e-01  7.05728325e-07 -2.90477420e-07 -2.43534526e-08]
 [ 6.75465625e-06  6.11445536e-01 -7.77834051e-01 -1.20764605e-05]
 [ 3.22160499e-06  7.77834051e-01  6.11445536e-01  2.39619011e-05]
 [-2.41312295e-08 -1.20834656e-05 -2.39468046e-05  9.99999214e-01]]

### The slow way

#### quantum map

A quantum map $\Lambda$ is the following.
$$
    \Lambda
    = \begin{pmatrix}
        \Lambda'(\ket{0}\bra{0}) & \cdots & \Lambda'(\ket{0}\bra{20}) \\
        \vdots & \ddots & \vdots \\
        \Lambda'(\ket{20}\bra{0}) & \cdots & \Lambda'(\ket{20}\bra{20})
    \end{pmatrix}
$$
$\Lambda'(\ket{i}\bra{j})$ is $20\times 20$ matrix, so $\Lambda$ is $20^2\times 20^2$ matrix.

In [102]:
∑.957phi = pi/4 # angle of rotation
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=15e3, atol=1e-8, rtol=1e-6)

# Quantum map
qmap = propagator(H_tot, T_g, c_op_list = [c_ops], args = {'phi': phi}, options = opt)

#### save quantum map

In [252]:
qsave(R,"results/rz"+str(phi))

#### Kraus map
$$
\Lambda(\rho) = \sum_{k=1}^N A_k\rho A_k^\dagger
$$
where $A_k$ are Kraus-operators

In [103]:
.957kraus_form = to_kraus(qmap)

#### Pauli transfer matrix
$$
(R_\Lambda)_{ij} = \frac{1}{d}\mathrm{Tr}[P_i\Lambda(P_j)] = \frac{1}{d}\sum_{k=1}^N\mathrm{Tr}[P_iA_kP_jA_k^\dagger]
$$

In [104]:
d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for i in range(d**2):
    for j in range(d**2):
        R[i,j] = 1/d * np.real(sum([(P[i]*A_k*P[j]*A_k.dag()).tr() for A_k in kraus_form]))
R = Qobj(R,dims=[[[2],[2]] for i in range(d)]) # Make quantum object

### $R_x(\theta)$-gate

In [370]:
T_g = 10/K # gate time
theta = 2*np.pi - 2*2.6893
theta_list = np.load("results/theta-list.npy")
delta_list = np.load("results/delta-list.npy")

def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect.bisect_right(a, x)
    if i:
        return i
    raise ValueError
.957
# find which delta that corresponds to a specific theta
z = find_le(theta_list,theta)
(x1,x2) = (delta_list[z-1], delta_list[z])
(y1,y2) = (theta_list[z-1], theta_list[z])
y = theta
x = (y-y1)*(x2-x1)/(y2-y1)+x1
delta = x

# detuning
def Delta(t,args):
    delta = args['delta']
    return delta * pow(np.sin(np.pi*t/T_g),2)

# Hamiltonian
H0 = - K * pow(a.dag(),2)*pow(a,2) + G * (pow(a.dag(),2) + pow(a,2))
H1 = - a.dag()*a
H_tot = [H0,[H1,Delta]]

In [377]:
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=25e3, atol=1e-8, rtol=1e-6)

d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
# Array with Pauli matrices
P = [I, sigma_x, -sigma_y, sigma_z]
for j in range(d**2):
    rho = mesolve(H_tot, P[j], [0,T_g] , c_ops, args = {'delta': delta}, options = opt)
    Lambda = rho.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * np.real((P[i]*Lambda).tr())
R = Qobj(R,dims=[[[2],[2]] for i in range(d)]) # Make quantum object

In [52]:
N = 2 # number of qubits
eye = qeye(2)
ket0 = basis(2,0)
ket1 = basis(2,1)
psi = (ket0)
rho = ket2dm(psi) # initial rho
vec_rho = 

In [379]:
qsave(R,"results/rx"+str(theta))

## $U(\Theta)$-gate

In [4]:
eye = qeye(num_lvl) # identity operator
T_g = 2 # gate time

a1 = tensor([a,eye])
a2 = tensor([eye,a])

# sigma z
sigma_z1 = tensor([sigma_z,eye])
sigma_z2 = tensor([eye,sigma_z])

# initial state
psi0 = tensor([up+down,up+down]).unit()

# coupling
def g(t,args):
    Theta = args['Theta']
    return np.pi*Theta/(8*T_g*pow(alpha,2))*np.sin(np.pi*t/T_g)
    
# Hamiltonian
H1 = K * pow(a1.dag(),2)*pow(a1,2) - G * (pow(a1.dag(),2) + pow(a1,2))
H2 = K * pow(a2.dag(),2)*pow(a2,2) - G * (pow(a2.dag(),2) + pow(a2,2))
H_coupling = a1.dag()*a2 + a2.dag()*a1
H_tot = [(H1+H2),[H_coupling,g]]

In [36]:
d = 4
Q = []
for i in range(d):
    Q.extend([tensor(P[i], P[j]) for j in range(d)])

In [9]:
Theta = 0.9046
gamma = 1/1500 # single-photon loss rate
c_ops = [np.sqrt(gamma)*a1,np.sqrt(gamma)*a2] # collapse operator
# For precise calculation
opt = Options(nsteps=15e3, atol=1e-8, rtol=1e-6)
rho = mesolve(H_tot, Q[0], [0,T_g] , c_ops, args = {'Theta': Theta}, options = opt)

In [227]:
Theta = 0.9046
gamma = 1/1500 # single-photon loss rate
c_ops = [np.sqrt(gamma)*a1,np.sqrt(gamma)*a2] # collapse operator

# For precise calculation
opt = Options(nsteps=15e3, atol=1e-8, rtol=1e-6)

d = 4
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for j in range(d**2):
    print(j)
    rho = mesolve(H_tot, Q[j], [0,T_g] , c_ops, args = {'Theta': Theta}, options = opt)
    Lambda = rho.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * np.real((Q[i]*Lambda).tr())
R = Qobj(R,dims=[[2 for i in range(d/2)] ,[2 for i in range(d/2)]]) #.957 Make quantum object

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


ValueError: too many values to unpack (expected 2)

#### $U(\Theta)$-gate (Constant amplitude)

In [7]:
eye = qeye(num_lvl) # identity operator
a1 = tensor([a,eye])
a2 = tensor([eye,a])

# sigma z
sigma_z1 = tensor([sigma_z,eye])
sigma_z2 = tensor([eye,sigma_z])

# initial state
psi0 = tensor([up+down,up+down]).unit()

# coupling
g = 0.1

# gate time
T_g = lambda Theta : Theta/(4*g*pow(alpha,2))
    
# Hamiltonian
H1 = - K * pow(a1.dag(),2)*pow(a1,2) + G * (pow(a1.dag(),2) + pow(a1,2))
H2 =  - K * pow(a2.dag(),2)*pow(a2,2) + G * (pow(a2.dag(),2) + pow(a2,2))
H_coupling = a1.dag()*a2 + a2.dag()*a1
H_tot = [(H1+H2)+H_coupling]

In [8]:
# Pauli matrices
d = 4
Q = []
for i in range(d):
    Q.extend([tensor(P[i], P[j]) for j in range(d)])

In [26]:
.957Theta = pi # one 10th of a degree
gamma = 1/1500 # single-photon loss rate
c_ops = [np.sqrt(gamma)*a1,np.sqrt(gamma)*a2] # collapse operator
# for precise calculation
opt = Options(nsteps=15e3)
rho = mesolve(H_tot, psi, [0,T_g(Theta)], c_ops, options = opt)

ValueError: operands could not be broadcast together with shapes (160000,) (16,) 

In [18]:
Theta = 2*pi/(2*360)*(2*180) # one half of a degree
gamma = 1/1500 # single-photon loss rate
c_ops = [np.sqrt(gamma)*a1,np.sqrt(gamma)*a2] # collapse operator
# For precise calculation
opt = Options(nsteps=15e3)
# pauli transfer matrix
R = np.zeros((d**2,d**2))

for j in range(d**2):
    rho = mesolve(H_tot, Q[j], [0,T_g(Theta)] , c_ops, options = opt)
    Lambda = rho.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * np.real((Q[i]*Lambda).tr())
X = Qobj(R,dims=[[2,2] for i in range(2)]) # Make quantum object

#### test

In [15]:
ket0 = basis(2,0)
ket1 = basis(2,1)
psi = tensor([(ket0+ket1).unit() for i in range(2)])
rho = ket2dm(psi) # initial rho
rho_vec = rho_to_pauli_basis(rho)
U = X
for i in range(180):
    U = U*X
rho_vec_final = X*rho_vec
rho_final = pauli_basis_to_rho(rho_vec_final)

NameError: name 'rho_to_pauli_basis' is not defined

In [24]:
rho_final

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.22875907+0.j         0.22930409-0.00756585j 0.22930409-0.00756585j
  0.22332079+0.j        ]
 [0.22930409+0.00756585j 0.23496389+0.j         0.23082443+0.j
  0.22930409+0.00756585j]
 [0.22930409+0.00756585j 0.23082443+0.j         0.23496389+0.j
  0.22930409+0.00756585j]
 [0.22332079+0.j         0.22930409-0.00756585j 0.22930409-0.00756585j
  0.22875907+0.j        ]]

In [13]:
rho_final

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.23946493+0.j         -0.23730753+0.02072756j -0.23730753+0.02072756j
   0.23695615+0.j        ]
 [-0.23730753-0.02072756j  0.23946841+0.j          0.2369604 +0.j
  -0.23730753-0.02072756j]
 [-0.23730753-0.02072756j  0.2369604 +0.j          0.23946841+0.j
  -0.23730753-0.02072756j]
 [ 0.23695615+0.j         -0.23730753+0.02072756j -0.23730753+0.02072756j
   0.23946493+0.j        ]]

#### Simulation with PTM

In [11]:
# Python Standard Library
from itertools import starmap, product

def rho_to_pauli_basis(rho):
    """
    Given a quantum operator write it in
    vector form in the Pauli basis.
    """
    dim = rho.shape[0]
    nq = int(np.log2(dim))
    dims = [[2]*nq, [1]*nq]
    data = np.zeros((dim**2,1),dtype=np.complex64)
    pauli_basis = (qeye(2), sigmax(), sigmay(), sigmaz())
    for idx, op in enumerate(starmap(tensor,product(pauli_basis, repeat=nq))):
        data[idx] = (op*rho).tr() / np.sqrt(dim)
    return Qobj(data, dims=dims)

def pauli_basis_to_rho(rho):
    """
    Given a quantum operator in vector written in the Pauli basis
    write it in the computational basis.
    """
    dim = rho.shape[0]
    nq = int(np.log2(np.sqrt(dim)))
    dims = [[2]*nq, [2]*nq]
    data = rho.data.toarray()
    rho = 0
    pauli_basis = (qeye(2), sigmax(), sigmay(), sigmaz())
    for idx, op in enumerate(starmap(tensor,product(pauli_basis, repeat=nq))):
        rho += (op*complex(data[idx]))*1/np.sqrt(2*nq)
    return Qobj(rho, dims=dims)

#### Pauli transfer matrix of identity

In [49]:
d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for j in range(d**2):
    pauli_basis = (qeye(2), sigmax(), sigmay(), sigmaz())
    Lambda = rx(0.9045853071795866) * pauli_basis[j] * rx(0.9045853071795866).dag()
    for i in range(d**2):
        R[i,j] = 1/d * np.real((pauli_basis[i]*Lambda).tr())
R = Qobj(R,dims=[[2,2] for i in range(d)]) # Make quantum object

<ipython-input-49-327fc0b7dad9>:6: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  Lambda = rx(0.9045853071795866) * pauli_basis[j] * rx(0.9045853071795866).dag()


#### Gates

In [347]:
def rz(phi):
    return (-1j*phi/2*sigmaz()).expm()

def rx(phi):
    return (1j*phi/2*sigmax()).expm()

def U(phi):
    sigmaz_1 = tensor(sigmaz(),qeye(2))
    sigmaz_2 = tensor(qeye(2),sigmaz())
    return (-1j*phi/2*sigmaz_1*sigmaz_2).expm()

ket0 = basis(2,0)
ket1 = basis(2,1)
psi = tensor([(ket0+ket1).unit() for i in range(2)])
rho = ket2dm(psi) # initial rho

In [350]:
psi_final = tensor(rx(0.9045853071795866),rx(0.9045853071795866))*tensor(rz(0.9046),qeye(2))*U(0.9046)*psi

In [351]:
ket2dm(psi_final)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.02319654e-09+0.00000000e+00j -1.71591071e-05+4.66367786e-06j
  -1.34921230e-05+1.81538118e-05j -8.33842708e-06+1.12198064e-05j]
 [-1.71591071e-05-4.66367786e-06j  3.09016728e-01+0.00000000e+00j
   3.09008389e-01-2.42944807e-01j  1.90975554e-01-1.50151135e-01j]
 [-1.34921230e-05-1.81538118e-05j  3.09008389e-01+2.42944807e-01j
   4.99999999e-01+0.00000000e+00j  3.09017209e-01-4.66367786e-06j]
 [-8.33842708e-06-1.12198064e-05j  1.90975554e-01+1.50151135e-01j
   3.09017209e-01+4.66367786e-06j  1.90983272e-01+0.00000000e+00j]]

#### Testing

In [50]:
N = 2 # number of qubits
eye = qeye(2)
ket0 = basis(2,0)
ket1 = basis(2,1)
psi = tensor([(ket0+ket1)/np.sqrt(2) for i in range(N)]) # initial state
rho = ket2dm(psi) # initial rho
vec_rho = rho_to_pauli_basis(rho)

In [51]:
RZ = qload("results/rz0.9046")
RX = qload("results/rx0.9045853071795866")
U = qload("results/U0.9046")

Loaded Qobj object:
Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isHerm = False

Loaded Qobj object:
Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isHerm = False

Loaded Qobj object:
Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (16, 16), type = super, isHerm = False



In [52]:
RZ = Qobj(RZ,dims=[[2],[2]],type='oper')
RX = Qobj(RX,dims=[[2],[2]],type='oper')
I = qeye(4)
I = Qobj(I, dims = [[2],[2]], type='oper')
U = Qobj(U,dims=[[2,2],[2,2]], type='oper')

In [53]:
vec_rho_final = tensor(RX,RX)*tensor(RZ,I)*U*vec_rho
rho_final = pauli_basis_to_rho(vec_rho_final)
rho_final

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.00911934+0.j          0.00688521-0.00235887j -0.00073071-0.00065487j
  -0.00523442+0.00258717j]
 [ 0.00688521+0.00235887j  0.30857269+0.j          0.2798707 -0.21895781j
   0.1801457 -0.14050671j]
 [-0.00073071+0.00065487j  0.2798707 +0.21895781j  0.4908149 +0.j
   0.28623766+0.00236963j]
 [-0.00523442-0.00258717j  0.1801457 +0.14050671j  0.28623766-0.00236963j
   0.19137943+0.j        ]]

In [56]:
U

Quantum object: dims = [[2, 2], [2, 2]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[ 9.99942955e-01  1.06995147e-05  0.00000000e+00  0.00000000e+00
   1.06995147e-05 -3.09173927e-08  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -4.59978800e-08 -5.39798213e-06
   0.00000000e+00  0.00000000e+00 -5.39798213e-06 -9.42052491e-07]
 [ 1.79904515e-05  6.11596449e-01  0.00000000e+00  0.00000000e+00
   2.00014182e-07  1.07677803e-05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -5.06847199e-04  5.32825871e-07
   0.00000000e+00  0.00000000e+00 -7.77647688e-01  5.12291784e-04]
 [ 0.00000000e+00  0.00000000e+00  6.11595759e-01 -1.05470962e-03
   0.00000000e+00  0.00000000e+00  1.07678562e-05  1.58481345e-08
  -9.63280268e-08  5.06847523e-04  0.00000000e+00  0.00000000e+00
   8.92800855e-06  7.77647688e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.05458954e-03  9.99942259e-01
   0.00000000e+00  0.00000000e+00 -9.099

In [54]:
ans = tensor(ket1,ket1)
ans.dag()*rho_final*ans

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.19137943]]